In [2]:
!java -version

java version "1.8.0_201"
Java(TM) SE Runtime Environment (build 1.8.0_201-b09)
Java HotSpot(TM) Client VM (build 25.201-b09, mixed mode)


In [3]:
!pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\09925296\appdata\local\continuum\miniconda3\lib\site-packages (19.0.3)


In [4]:
!python -V

Python 3.7.0


In [5]:
!pip install z:/data/JPype1-0.6.3-cp37-cp37m-win32.whl

In [6]:
!pip install konlpy

In [7]:
from konlpy.tag import Hannanum

In [8]:
han = Hannanum()

In [9]:
from konlpy.tag import Kkma

In [10]:
kkma = Kkma()

In [12]:
print(han.analyze(u'롯데마트의 흑마늘 양념 치킨이 논란이 되고 있다.'))

[[[('롯데마트', 'ncn'), ('의', 'jcm')], [('롯데마트의', 'ncn')], [('롯데마트', 'nqq'), ('의', 'jcm')], [('롯데마트의', 'nqq')]], [[('흑마늘', 'ncn')], [('흑마늘', 'nqq')]], [[('양념', 'ncn')]], [[('치킨', 'ncn'), ('이', 'jcc')], [('치킨', 'ncn'), ('이', 'jcs')], [('치킨', 'ncn'), ('이', 'ncn')]], [[('논란', 'ncpa'), ('이', 'jcc')], [('논란', 'ncpa'), ('이', 'jcs')], [('논란', 'ncpa'), ('이', 'ncn')]], [[('되', 'nbu'), ('고', 'jcj')], [('되', 'nbu'), ('이', 'jp'), ('고', 'ecc')], [('되', 'nbu'), ('이', 'jp'), ('고', 'ecs')], [('되', 'nbu'), ('이', 'jp'), ('고', 'ecx')], [('되', 'paa'), ('고', 'ecc')], [('되', 'paa'), ('고', 'ecs')], [('되', 'paa'), ('고', 'ecx')], [('되', 'pvg'), ('고', 'ecc')], [('되', 'pvg'), ('고', 'ecs')], [('되', 'pvg'), ('고', 'ecx')], [('되', 'px'), ('고', 'ecc')], [('되', 'px'), ('고', 'ecs')], [('되', 'px'), ('고', 'ecx')]], [[('있', 'paa'), ('다', 'ef')], [('있', 'px'), ('다', 'ef')]], [[('.', 'sf')], [('.', 'sy')]]]


In [13]:
from urllib.request import urlopen
from urllib.parse import quote_plus
from bs4 import BeautifulSoup as soup
from newspaper import Article
from time import sleep
import time
import pandas as pd
import numpy as np

In [14]:
# naver news 기간검색
def naver_search_d(qry,beg,end,pages=3): 
    articles = []
    title = []
    href = []
    date = []
    text = []
    stime = time.time()
    # original 주소 
    url0 = "https://search.naver.com/search.naver?&where=news&query={0}&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={1}&de={2}&docid=&nso=so:r,p:from{3}to{4},a:all&mynews=0&cluster_rank=22&start={5}&refresh_start=0"
    # 네이버 검색결과 page별
    begx = beg.replace('.','')
    endx = end.replace('.','')
    for i in range(pages): 
        start = i*10+1
        qry1 = quote_plus(qry)
        url = url0.format(qry1,beg,end,begx,endx,start)
        obj = soup(urlopen(url),'html.parser')
        # links : naver 검색결과 뉴스들의 링크들
        links = obj.find_all('a',{'class':'_sp_each_title'})
        # 링크별 기사가져오기 
        for link in links:
            h = link.get('href') # hyperlink
            try:
                a = Article(h,language='ko') # 기사가져오기 
                a.download();a.parse()
                articles.append(a)
                title.append(a.title)
                date.append(a.publish_date)
                text.append(a.text)
            except:
                pass
            sleep(np.random.randint(1,4))
    etime = time.time() - stime
    elaps = time.strftime('%H:%M:%S',time.gmtime(etime))
    news = pd.DataFrame(dict(qry=qry,date=date,title=title,text=text))        
    print('{} {} search done, {} elapsed'.format(qry,begx[:4],elaps))
    return news
# naver_search_d

In [32]:
%%time
drng = lambda x,y,f: pd.date_range(x,y,freq=f).strftime('%Y.%m.%d')
frm = drng('2013','2019','YS')
tom = drng('2013','2019','Y')
news = []
for f,t in zip(frm,tom):
    news.append(naver_search_d('보험계약대출+약관대출',f,t,pages=2))

보험계약대출+약관대출 2013 search done, 00:00:54 elapsed
보험계약대출+약관대출 2014 search done, 00:00:58 elapsed
보험계약대출+약관대출 2015 search done, 00:00:47 elapsed
보험계약대출+약관대출 2016 search done, 00:00:40 elapsed
보험계약대출+약관대출 2017 search done, 00:00:53 elapsed
보험계약대출+약관대출 2018 search done, 00:00:46 elapsed
Wall time: 5min


In [33]:
newsdf = pd.concat(news,axis=0)

In [34]:
newsdf.shape

(120, 4)

In [35]:
newsdf.head()

,qry,date,title,text
0,보험계약대출+약관대출,None,2014 새해 달라지는 것들-금융ㆍ증권,◇자동차보험 차량모델등급제도 개선 = 보험료의 공평한 부담을 위해 자가용 승용차의 ...
1,보험계약대출+약관대출,None,2014년부터 달라지는 은행·보험·증권 금융제도,▲(사진=왕진오 기자)\n\n내년부터 ATM 현금거래시 마그네틱카드 사용이 전면 금...
2,보험계약대출+약관대출,2013-12-22 12:00:00+09:00,10월말 보험사 대출 125.6조원..연체율 0.69%,[이데일리 이준기 기자] 금융감독원은 10월말 보험회사의 대출채권 잔액이 125조6...
3,보험계약대출+약관대출,None,대체휴일제 추석때 첫 적용…운전중 DMB 보면 범칙금,\n\n\n\n\n\n\n\n◆ 새해 달라지는 것 ◆◆ 어르신 임플란트 보험급여 적...
4,보험계약대출+약관대출,None,<새해 달라지는 것> 금융·증권,▲자동차보험 차량모델등급제도 개선 = 보험료의 공평한 부담을 위해 자가용 승용차의 ...


In [36]:
from konlpy.tag import Kkma

In [37]:
kkma = Kkma()

In [39]:
nouns = newsdf.text.apply(lambda x: kkma.nouns(x))

In [41]:
newsdf['nouns'] = nouns

In [42]:
import nltk

In [45]:
from nltk.tokenize import sent_tokenize, word_tokenize